In [22]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score
from meteostat import Point, Hourly, units
from itertools import combinations
import random

In [23]:
data = pd.read_csv('la_data.csv')

In [24]:
display(data.head(3))
print('Shape of original data: \n', data.shape)
print('\n')

data.info()
print('\n')

print('Number of columns of each data type: \n', data.dtypes.value_counts())
print('\n')

isnull_series = data.isnull().sum()[data.isnull().sum() > 0]
print('Columns with null values: \n', isnull_series.sort_values(ascending=False))

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-5236,2,2016-03-22 19:36:44,2016-03-23 01:36:44,34.09256,-118.20622,34.09256,-118.20622,0.000,At Avenue 43 - Accident.,...,False,False,True,False,False,False,Night,Night,Day,Day
1,A-5239,2,2016-03-22 20:59:43,2016-03-23 02:59:43,33.94819,-118.27973,33.94676,-118.27975,0.099,At Century Blvd - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-5254,3,2016-03-23 07:59:47,2016-03-23 13:59:47,34.02330,-118.17288,34.02138,-118.17339,0.136,At Whittier Blvd/Olympic Blvd - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day


Shape of original data: 
 (68956, 47)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68956 entries, 0 to 68955
Data columns (total 47 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     68956 non-null  object 
 1   Severity               68956 non-null  int64  
 2   Start_Time             68956 non-null  object 
 3   End_Time               68956 non-null  object 
 4   Start_Lat              68956 non-null  float64
 5   Start_Lng              68956 non-null  float64
 6   End_Lat                68956 non-null  float64
 7   End_Lng                68956 non-null  float64
 8   Distance(mi)           68956 non-null  float64
 9   Description            68956 non-null  object 
 10  Number                 22123 non-null  float64
 11  Street                 68956 non-null  object 
 12  Side                   68956 non-null  object 
 13  City                   68956 non-null  object 
 14  County        

In [25]:
print('All Columns from the Original Data Set: \n', data.columns)

drop_cols = []

drop_cols.append('ID') # ID column is not needed either
drop_cols.append('Severity') # we will ignore severity here and treat them all as positive data
drop_cols.append('End_Time')
# drop_cols.append('Start_Lat')
# drop_cols.append('Start_Lng')
drop_cols.append('End_Lat')
drop_cols.append('End_Lng')
drop_cols.append('Distance(mi)')
drop_cols.append('Description')
drop_cols.append('Number')
drop_cols.append('Street') 
drop_cols.append('Side') 
drop_cols.append('City') 
drop_cols.append('County') 
drop_cols.append('State') 
drop_cols.append('Zipcode')
drop_cols.append('Country')
drop_cols.append('Timezone')
drop_cols.append('Airport_Code')
drop_cols.append('Weather_Timestamp')
drop_cols.append('Wind_Chill(F)') 
drop_cols.append('Visibility(mi)')
drop_cols.append('Wind_Direction')
drop_cols.append('Weather_Condition')
drop_cols.append('Sunrise_Sunset')
drop_cols.append('Civil_Twilight')
drop_cols.append('Nautical_Twilight')
drop_cols.append('Astronomical_Twilight')

print('Finalized columns to drop', drop_cols)


All Columns from the Original Data Set: 
 Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number', 'Street',
       'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')
Finalized columns to drop ['ID', 'Severity', 'End_Time', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number', 'Street', 'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_

In [26]:
la_df = data.drop(drop_cols, axis=1, inplace=False)
display(la_df.head())
display(la_df.tail())

,Start_Time,Start_Lat,Start_Lng,Temperature(F),Humidity(%),Pressure(in),Wind_Speed(mph),Precipitation(in),Amenity,Bump,...,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop
0,2016-03-22 19:36:44,34.09256,-118.206220,64.0,24.0,30.00,9.2,NaN,False,False,...,False,False,False,False,False,False,True,False,False,False
1,2016-03-22 20:59:43,33.94819,-118.279730,64.0,23.0,30.02,11.5,NaN,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2016-03-23 07:59:47,34.02330,-118.172880,55.9,53.0,30.11,NaN,NaN,False,False,...,False,True,False,False,False,False,False,False,False,False
3,2016-03-23 11:50:32,34.14470,-118.278650,73.9,14.0,30.15,NaN,NaN,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2016-03-23 12:16:45,34.09914,-118.251853,73.9,14.0,30.15,NaN,NaN,False,False,...,False,False,False,False,False,False,False,False,False,False


,Start_Time,Start_Lat,Start_Lng,Temperature(F),Humidity(%),Pressure(in),Wind_Speed(mph),Precipitation(in),Amenity,Bump,...,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop
68951,2019-08-22 17:07:14,34.03693,-118.43877,72.0,76.0,29.57,8.0,0.0,False,False,...,False,True,False,False,False,False,False,False,False,False
68952,2019-08-23 04:04:48,34.07579,-118.27668,67.0,79.0,29.62,0.0,0.0,False,False,...,False,False,False,False,False,False,False,False,False,False
68953,2019-08-23 12:52:31,34.02379,-118.27639,81.0,49.0,29.66,0.0,0.0,False,False,...,False,False,False,False,False,False,False,False,False,False
68954,2019-08-23 13:42:50,34.07061,-118.26391,82.0,47.0,29.65,0.0,0.0,False,False,...,False,True,False,False,False,False,False,False,False,False
68955,2019-08-23 15:45:43,34.04365,-118.44373,74.0,66.0,29.64,8.0,0.0,False,False,...,False,True,False,False,False,False,False,False,False,False


In [27]:
print(la_df.shape)
la_df.dropna(subset=['Start_Time'], inplace=True) # drop rows that have null start_time
la_df['Start_Time'] = pd.to_datetime(la_df['Start_Time']) # transform to datetime 
print(la_df.shape)

(68956, 21)
(68956, 21)


In [28]:
la_df['Target']= 1 # the original data is all positive data
la_df.shape

(68956, 22)

In [29]:
# For random generation of 'Start_Time' df.loc[len(df.index)] = ['Amy', 89, 93] 
year_list = ['2016', '2017', '2018', '2019']
mon_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
year_mon_list = []
for y in year_list:
    for m in mon_list:
        year_mon_list.append(y+'-'+m)

remove_items = ['2016-01', '2016-02', '2019-09', '2019-10', '2019-11', '2019-12']

for i in remove_items:
    year_mon_list.remove(i)
    
year_mon_list
short_mon = ['04', '06', '09', '11']

In [30]:
def generate_rand_datetime(year_mon_list):
    rand_y_m = random.choice(year_mon_list)
    if rand_y_m[-2:] == '02':
        rand_d = random.randint(1, 28)
    elif rand_y_m[-2:] in short_mon:
        rand_d = random.randint(1, 30)
    else: 
        rand_d = random.randint(1, 31)

    rand_h = random.randint(0, 23)

    rand_datetime = rand_y_m + '-' + str(rand_d) + ' ' + str(rand_h) + ':00:00'
    
    return rand_datetime


In [31]:
def get_weather_info(start,lat, lng):
    
    location = Point(lat, lng)
    
    time = dt.datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
    data = Hourly(location, time, time)
    data = data.convert(units.imperial)
    data = data.fetch()
    
    return data['temp'][0], data['rhum'][0], data['pres'][0], data['wspd'][0], data['prcp'][0]


In [32]:
def generate_neg_rows(df):
    for i in range(len(df)):
        ### Add 1st new negative row
        new1 = df.iloc[i].tolist()
        new1[0] = generate_rand_datetime(year_mon_list) # generate a random datetime
        new1[3], new1[4], new1[5], new1[6], new1[7] = get_weather_info(new1[0], new1[1], new1[2]) # replace weather info
        new1[-1] = 0 # assign negative label
        df.loc[len(df.index)] = new1 
    
        ### Add 2nd new negative row
        new2 = df.iloc[i].tolist()
        new2[0] = generate_rand_datetime(year_mon_list)
        new2[3], new2[4], new2[5], new2[6], new2[7] = get_weather_info(new2[0], new2[1], new2[2]) # replace weather info
        new2[-1] = 0
        df.loc[len(df.index)] = new2 
    
        ### Add 3rd new negative row
        new3 = df.iloc[i].tolist()
        new3[0] = generate_rand_datetime(year_mon_list)
        new3[3], new3[4], new3[5], new3[6], new3[7] = get_weather_info(new3[0], new3[1], new3[2])
        new3[-1] = 0
        df.loc[len(df.index)] = new3
    
    return df
    

In [33]:
la_df_full = generate_neg_rows(la_df) 

In [34]:
la_df_full.shape

(275824, 22)

In [35]:
la_df_full.to_csv('la_data_full.csv', index=False)

In [ ]:
# date = "2022-3-5 20:00:00"
# date2 = dt.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
# print(date2)

# print(get_weather_info(date, 34.092, -118.206))

In [ ]:
la_df.shape